In [3]:
from train_model import train_model
from model import ShakespearModel

N_EPOCHS = 2
N_TOKENS = 250  # N
N_LAYERS = 6  # L
N_HEADS = 4  # h
N_WORKERS = 2
BATCH_SIZE = 5  # B
D_MODEL = 200  # d
D_K = 32
D_V = D_K
D_FF = 512
RAW_DATA_PATH = './datasets/shakespear_corpus.txt'

#trained_mod = train_model(N_EPOCHS, N_TOKENS, N_LAYERS, N_HEADS, BATCH_SIZE, D_MODEL, D_K, D_V, D_FF)

In [4]:
from train_model import load_data
from parsing.CharDataSet import CharDataSet
from torch.utils.data.dataloader import DataLoader
import torch
import pandas as pd

raw_data = load_data(RAW_DATA_PATH)

tokenized_data = CharDataSet(N_TOKENS, raw_data)
print(tokenized_data.encode(raw_data))
data_loader = DataLoader(
    tokenized_data,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
)

trained_mod = ShakespearModel(N_LAYERS, N_HEADS, D_MODEL, D_FF, D_K, D_V, BATCH_SIZE, N_TOKENS, tokenized_data.get_vocab_size())
print(trained_mod.param_norm())

tensor([[18., 47., 56.,  ..., 45.,  8.,  0.]])
tensor(135.2915)


In [8]:
# define a cross validation function
def crossvalid(args,k_fold=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    train_score = pd.Series()
    val_score = pd.Series()
    
    total_size = len(dataset)
    fraction = 1/k_fold
    seg = int(total_size * fraction)
    # tr:train,val:valid; r:right,l:left;  eg: trrr: right index of right side train subset 
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for i in range(k_fold):
        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
        print("train indices: [%d,%d),[%d,%d), test indices: [%d,%d)" % (trll,trlr,trrl,trrr,vall,valr))
        
        train_left_indices = list(range(trll,trlr))
        train_right_indices = list(range(trrl,trrr))
        
        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall,valr))
        
        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset,val_indices)
        
        print(len(train_set),len(val_set))
        print()
        
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=N_WORKERS)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=N_WORKERS)
        train_acc = train(res_model,criterion,optimizer,train_loader,epoch=1)
        train_score.at[i] = train_acc
        val_acc = valid(res_model,criterion,optimizer,val_loader)
        val_score.at[i] = val_acc
        val_score = 0 #! to remove
    
    return train_score,val_score

In [9]:
train_score,val_score = crossvalid(dataset=tokenized_data)

train indices: [0,0),[223028,1115144), test indices: [0,223028)
892116 223028

train indices: [0,223028),[446056,1115144), test indices: [223028,446056)
892116 223028

train indices: [0,446056),[669084,1115144), test indices: [446056,669084)
892116 223028

train indices: [0,669084),[892112,1115144), test indices: [669084,892112)
892116 223028

train indices: [0,892112),[1115140,1115144), test indices: [892112,1115140)
892116 223028



In [7]:
print(train_score)
print(val_score)

0    ShakespearModel(\n  (WPE): WPE()\n  (WTE): Emb...
1    ShakespearModel(\n  (WPE): WPE()\n  (WTE): Emb...
2    ShakespearModel(\n  (WPE): WPE()\n  (WTE): Emb...
3    ShakespearModel(\n  (WPE): WPE()\n  (WTE): Emb...
4    ShakespearModel(\n  (WPE): WPE()\n  (WTE): Emb...
dtype: object
0


In [3]:
seed = "O God, O God!"
idx = tokenized_data.encode(seed)

In [4]:
print(trained_mod.param_norm())
new_tokens = trained_mod.generate(idx, n_new_tokens=50)
print(trained_mod.param_norm())
print(new_tokens)

tensor(134.6081)
idx tensor([[27.,  1., 19., 53., 42.,  6.,  1., 27.,  1., 19., 53., 42.,  2.]])
tensor(134.6081)
tensor([27.,  1., 19., 53., 42.,  6.,  1., 27.,  1., 19., 53., 42.,  2., 15.,
        15., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 30., 30., 30.,
        30., 30., 30., 11., 11., 30., 30., 11., 11., 11., 11., 11., 11., 30.,
        30., 30., 30., 30., 30., 29., 29., 29., 29., 29., 29., 29., 29., 29.,
        29., 30., 30., 30., 30., 29., 11.])


In [5]:
print(tokenized_data.decode(new_tokens))

O God, O God!CC;;;;;;;;;;RRRRRR;;RR;;;;;;RRRRRRQQQQQQQQQQRRRRQ;


In [6]:
for name, param in trained_mod.named_parameters():
    print(name, param.size())

WTE.weight torch.Size([65, 200])
transformer.blocks.0.CausalSelfAttn.to_Q.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_Q.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.to_K.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_K.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.to_V.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_V.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.W_O.weight torch.Size([200, 128])
transformer.blocks.0.CausalSelfAttn.W_O.bias torch.Size([200])
transformer.blocks.0.LayerNorm_1.weight torch.Size([200])
transformer.blocks.0.LayerNorm_1.bias torch.Size([200])
transformer.blocks.0.FFN.L1.weight torch.Size([512, 200])
transformer.blocks.0.FFN.L1.bias torch.Size([512])
transformer.blocks.0.FFN.L2.weight torch.Size([200, 512])
transformer.blocks.0.FFN.L2.bias torch.Size([200])
transformer.blocks.0.LayerNorm_2.weight torch.Size([200])
transformer.blocks.0.LayerNorm_2.bias t

In [ ]:
for name, param in trained_mod.transformer.blocks[0].CausalSelfAttn.named_parameters():
    print(name, param)